In [1]:
import pandas as pd
from causallearn.utils.FastKCI.FastKCI import FastKCI_CInd, FastKCI_UInd
from causallearn.utils.KCI.KCI import KCI_CInd, KCI_UInd
from causallearn.utils.RCIT.RCIT import RIT, RCIT
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern
from hyppo.independence import Hsic
from src.fcit.fcit import fcit
from prettytable import PrettyTable

In [9]:
def instrumental_variable(A, W):
    iv_model = RandomForestRegressor().fit(y=A.squeeze(), X=W)
    A_hat = iv_model.predict(W)
    return np.expand_dims(A_hat, axis=1)

class FIT(object):
    def __init__(self):
        pass
    def compute_pvalue(self, x,y, z=None):
        p_value = fcit.test(x,y,z)
        return (p_value, None)
    
class GP_HSIC(object):
    def __init__(self):
        pass
    def compute_pvalue_noniid(self, x,y,s,z=None):
        normalize = True
        # GP regressions of X,Y,Z onto S to obtain residuals
        kernel = Matern(nu=1.5)
        gp_x = GaussianProcessRegressor(kernel=kernel, normalize_y=normalize)
        gp_x.fit(s, x)
        r_x = x - gp_x.predict(s)
        gp_y = GaussianProcessRegressor(kernel=kernel, normalize_y=normalize)
        gp_y.fit(s, y)
        r_y = y -  gp_y.predict(s)

        
        if z is not None:
            #kernel = C(1.0) * RBF(length_scale=1.0) # TODO, how to choose kernel 
            # GP regression of residuals onto Z to obtain additional residuals
            gp_z = GaussianProcessRegressor(kernel=kernel, normalize_y=normalize)
            gp_z.fit(s, z)
            r_z = z - gp_z.predict(s)
        
            gp_rxz = GaussianProcessRegressor(kernel=kernel, normalize_y=normalize)
            gp_rxz.fit(r_z, r_x)
            r_xz = r_x - gp_rxz.predict(r_z)
            gp_ryz = GaussianProcessRegressor(kernel=kernel, normalize_y=normalize)
            gp_ryz.fit(r_z, r_y)
            r_yz = r_y - gp_ryz.predict(r_z)
        else:
            # If not conditional, resort to normal independece test of residuals
            r_xz = r_x
            r_yz = r_y
            
        # HSIC test of residuals
        stat, pval = Hsic().test(r_xz, r_yz)
        return (pval, stat)
        

    def compute_pvalue(self, x,y,z=None):
        kernel = C(1.0) * RBF(length_scale=1.0) # TODO, how to choose kernel 
        
        if z is not None:
            # GP regression to obtain residuals of X,Y onto Z
            gp_xz = GaussianProcessRegressor(kernel=kernel, normalize_y=True,)
            gp_xz.fit(z, x)
            r_x = x - gp_xz.predict(z)
            gp_yz = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
            gp_yz.fit(z, y)
            r_y = y - gp_yz.predict(z)
        else:
            # If not conditional, revert to regular independence test
            r_x = x
            r_y = y
            
        # HSIC test for independence
        stat, pval = Hsic().test(r_x, r_y)
        return (pval, stat)    
    
def run_evidence_test(df, edge, evidence, test='randomized', iv=False):
    test_indirect, test_direct = None, None
    if len(evidence) == 3:
        if test == 'RCIT':
            test_indirect = RCIT()
            test_direct = RIT()
        elif test == 'KCIT':
            test_indirect = KCI_CInd()
            test_direct = KCI_UInd()
        elif test== 'FastKCIT':
            test_indirect = FastKCI_CInd()
            test_direct = FastKCI_UInd()
        elif test == 'FIT':
            test_indirect = FIT()
            test_direct = FIT()
        elif test == "GP_HSIC":
            test_indirect = GP_HSIC()
            test_direct = GP_HSIC()
        else:
            print(f"Unsupported CI test {test}")
        A = edge[0]
        B = edge[1]
        I = evidence[1]
        d_A = np.expand_dims(df[A].to_numpy(), axis=1)
        d_B =  np.expand_dims(df[B].to_numpy(), axis=1)
        d_I =  np.expand_dims(df[I].to_numpy(), axis=1)
        if iv:
            d_W = np.expand_dims(df['radiation'].to_numpy(), axis=1)
            if test == 'GP_HSIC':
                p_direct = test_direct.compute_pvalue_noniid(x=d_A, y=d_B, s=d_W)[0]
                p_indirect = test_indirect.compute_pvalue_noniid(x=d_A, y=d_B, z=d_I, s=d_W)[0] 
            else:
                d_A = instrumental_variable(d_A, d_W)
                p_direct = test_direct.compute_pvalue(d_A, d_B)[0]
                p_indirect = test_indirect.compute_pvalue(d_A, d_B, d_I)[0]
        else:
            p_direct = test_direct.compute_pvalue(d_A, d_B)[0]
            p_indirect = test_indirect.compute_pvalue(d_A, d_B, d_I)[0]    
        
        # if p_direct <=0.05:
        #     test_str = f"Test failed, {A} is dependent to {B} conditioned on the empty set: {p_direct}\n"
        # else:
        #     test_str = f"Test passed, {A} is independent of {B} conditioned on the empty set: {p_direct}\n"
        # if p_indirect <=0.05:
        #     test_str += f"Test failed, {A} is dependent to {B} conditioned on {I}: {p_indirect}"
        # else:
        #     test_str += f"Test passed, {A} is independent of {B} conditioned on {I}: {p_indirect}"
        direct_test = "pass" if p_direct > 0.05 else "fail"
        indirect_test = "pass" if p_indirect > 0.05 else "fail"
        return direct_test, round(p_direct,4), indirect_test, round(p_indirect, 4)
    else:
        return None

In [3]:
celltype='huvec'
doses = ["A", "B", "C", "D", "E"]
dfs = [pd.read_csv(f"./data/{celltype}/cd_matrix_d{d}.csv", header=0) for d in doses]
df_map = dict(zip(doses, dfs))
df_consensus_w_edges =  pd.read_csv(f"./data/{celltype}/consensus_edges_w_indirect_evidence.csv", header=0)

In [10]:
ci_tests = ["KCIT",  "RCIT", "GP_HSIC"] #"FastKCIT",
iv_variable = [True, False]
for ci_test in ci_tests:
    for iv in iv_variable:
        table=PrettyTable()
        table.field_names = ["Edge (A,B)", 
                            "Evidence (A,I,B)", 
                            "A indep B (pvalue)", "A indep B | I (pvalue)", "Support"]
        table.title = f"{ci_test} (IV radiation: {iv})"
        for _, row in df_consensus_w_edges.iterrows():
            if row['evidence'] != 'true positive':
                edge = (row['TF'], row['target'])
                evidence = row['evidence'].replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
                df = df_map[row['dose']]
                output = run_evidence_test(df, edge, evidence, test=ci_test, iv=iv)
                if output is not None:
                    test_direct, direct_pvalue, test_indirect, indirect_pvalue = output
                    if (test_direct=="fail" and test_indirect == "pass"):
                        support = "indirect"
                    elif (test_direct=="fail" and test_indirect == "fail"):
                        support = "direct"
                    else:
                        support = "inconclusive"
                    table.add_row([edge, evidence,f"{test_direct} ({direct_pvalue})", f"{test_indirect} ({indirect_pvalue})", support])
        print(table)
            


+--------------------------------------------------------------------------------------------------------------+
|                                          KCIT (IV radiation: True)                                           |
+--------------------+----------------------------+--------------------+------------------------+--------------+
|     Edge (A,B)     |      Evidence (A,I,B)      | A indep B (pvalue) | A indep B | I (pvalue) |   Support    |
+--------------------+----------------------------+--------------------+------------------------+--------------+
|  ('EGR2', 'CFL1')  |  ['EGR2', 'TP53', 'CFL1']  |   pass (0.5583)    |     pass (0.1402)      | inconclusive |
|  ('EGR2', 'ID1')   |  ['EGR2', 'TP53', 'ID1']   |   pass (0.5151)    |      pass (0.285)      | inconclusive |
|  ('EGR2', 'MMP2')  |  ['EGR2', 'TP53', 'MMP2']  |   pass (0.6265)    |     pass (0.2644)      | inconclusive |
|  ('EGR2', 'EEF2')  |  ['EGR2', 'TP53', 'EEF2']  |   pass (0.1629)    |     pass (0.3533)      